# 07. 범주형 데이터 이진 분류
## 7.3 베이스라인 모델
### 7.3.1 피처 엔지니어링
### 7.3.2 모델 훈련
### 7.3.3 모델 성능 검증
### 7.3.4 예측 및 결과 제출

### 7.3.1 피처 엔지니어링

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_path = '../../data/07_categorical/'

train = pd.read_csv(data_path+'train.csv', index_col='id')
test = pd.read_csv(data_path+'test.csv', index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

In [4]:
# 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1)
all_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [5]:
# 원핫 인코딩
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)

In [8]:
# 데이터 나누기
num_train = len(train)

X_train_o = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_o,
    y,
    test_size=0.1,
    stratify=y, # train과 valid의 타깃 비율을 y와 같은 비율로 나눠줌
    random_state=10
)

### 7.3.2 모델 훈련

In [10]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

### 7.3.3 모델 성능 검증

In [11]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [12]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)
print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7965


### 7.3.4 예측 및 결과 제출

In [13]:
y_preds = logistic_model.predict_proba(X_test)[:, 1]

In [14]:
submission['target'] = y_preds
submission.to_csv(data_path + 'submission.csv')